# Building powerful image classification models using very little data

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import os
import datetime as dt
import numpy as np
import pandas as pd

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [6]:
from keras import backend as K
K.set_image_dim_ordering('th')

### Image data generation from transformations

In [7]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [8]:
img = load_img('data/train/cats/cat.0.jpg')

In [9]:
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

In [10]:
i = 0
for batch in datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break

### Build small convnet

In [11]:
model = Sequential()

In [12]:
# input conv2d layer layer 32 filters, 3x3 kernel
model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# conv2d layer 32 filters, 3x3 kernel
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# conv2d layer 64 filters, 3x3 kernel
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [13]:
# this converts our 3D feature maps to 1D feature vectors
model.add(Flatten())

# fully connected layer, 64 outputs
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))

# output layer and sigmoid activation
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Set up data generators

In [15]:
batch_size = 16
nb_train_samples = 4000
nb_validation_samples = 800
epochs = 50

In [16]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [17]:
# this is the augmentation configuration we will use for testing (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
# this is a generator that will read pictures found in subfolders of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',            # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')     # since we use binary_crossentropy loss, we need binary labels

Found 4000 images belonging to 2 classes.


In [19]:
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 800 images belonging to 2 classes.


### Train model

In [20]:
model_history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
model.save_weights('first_try.h5')

Epoch 1/50
250/250 [==============================] - 500s 2s/step - loss: 0.6863 - acc: 0.5522 - val_loss: 0.6667 - val_acc: 0.5475
Epoch 2/50
250/250 [==============================] - 519s 2s/step - loss: 0.6516 - acc: 0.6348 - val_loss: 0.5910 - val_acc: 0.6837
Epoch 3/50
250/250 [==============================] - 528s 2s/step - loss: 0.6102 - acc: 0.6775 - val_loss: 0.5706 - val_acc: 0.7025
Epoch 4/50
250/250 [==============================] - 524s 2s/step - loss: 0.5925 - acc: 0.6957 - val_loss: 0.5571 - val_acc: 0.7375
Epoch 5/50
250/250 [==============================] - 484s 2s/step - loss: 0.5593 - acc: 0.7158 - val_loss: 0.5255 - val_acc: 0.7462
Epoch 6/50
250/250 [==============================] - 466s 2s/step - loss: 0.5506 - acc: 0.7238 - val_loss: 0.5297 - val_acc: 0.7475
Epoch 7/50
250/250 [==============================] - 264s 1s/step - loss: 0.5392 - acc: 0.7445 - val_loss: 0.5333 - val_acc: 0.7350
Epoch 8/50
250/250 [==============================] - 299s 1s/step - 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jeremysmith/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-8a58e343e8e9>", line 6, in <module>
    validation_steps=nb_validation_samples // batch_size)
  File "/Users/jeremysmith/anaconda3/lib/python3.6/site-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/Users/jeremysmith/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1418, in fit_generator
    initial_epoch=initial_epoch)
  File "/Users/jeremysmith/anaconda3/lib/python3.6/site-packages/keras/engine/training_generator.py", line 217, in fit_generator
    class_weight=class_weight)
  File "/Users/jeremysmith/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1217, in train_on_batch
    outputs = self.train_function(ins)
  File "/Users/jeremysmith/anaconda3/lib/python3

KeyboardInterrupt: 

In [21]:
pd.DataFrame(model_history.history).plot();

NameError: name 'model_history' is not defined